## Week 2 Day 2

Our first Agentic Framework project!!

Prepare yourself for something ridiculously easy.

We're going to build a simple Agent system for generating cold sales outreach emails:
1. Agent workflow
2. Use of tools to call functions
3. Agent collaboration via Tools and Handoffs

## Before we start - some setup:


Please visit Sendgrid at: https://sendgrid.com/

(Sendgrid is a Twilio company for sending emails.)

If SendGrid gives you problems, see the alternative implementation using "Resend Email" in community_contributions/2_lab2_with_resend_email

Please set up an account - it's free! (at least, for me, right now).

Once you've created an account, click on:

Settings (left sidebar) >> API Keys >> Create API Key (button on top right)

Copy the key to the clipboard, then add a new line to your .env file:

`SENDGRID_API_KEY=xxxx`

And also, within SendGrid, go to:

Settings (left sidebar) >> Sender Authentication >> "Verify a Single Sender"  
and verify that your own email address is a real email address, so that SendGrid can send emails for you.


In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio

## Step 1: Agent workflow

In [69]:
instructions1 = "You are a customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."


instructions3 = "You are a busy customer service agent working for BroomBot (Motorcycle Bot), \
a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. \
You write concise, to the point cold emails."

In [87]:
customer_service_agent1 = Agent(
        name="Professional Customer Service Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

customer_service_agent2 = Agent(
        name="Engaging Customer Service Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

customer_service_agent3 = Agent(
        name="Busy Customer Service Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:

result = Runner.run_streamed(customer_service_agent1, input="Write a cold customer service email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Discover BroomBot's Premier Motorcycle Services

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with BroomBot, a leader in AI-powered motorcycle solutions. We specialize in providing exceptional motorcycle services, from booking to purchasing, designed to enhance your riding experience.

At BroomBot, we understand the importance of dependable transportation and the joy of riding. Our services include:

- **Motorcycle Booking Service**: Effortlessly schedule rides and access our fleet of well-maintained motorcycles at your convenience.
- **Sales of Premium Motorcycles**: Choose from a wide range of models, ensuring you find the perfect fit for your lifestyle and preferences.

I would love the opportunity to discuss how BroomBot can meet your specific needs. Please feel free to reach out if you have any questions or if you would like to explore our services further.

Thank you for considering BroomBot. I look forward to the possibi

In [ ]:
messages = "write a cold customer service email"

with trace("Parallel cold email"):
    results = await asyncio.gather(
        Runner.run(customer_service_agent1, messages),
        Runner.run(customer_service_agent2, messages),
        Runner.run(customer_service_agent3, messages)
    )
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

<!DOCTYPE html>  
<html lang='en'>  
<body>  
<p>Dear [Customer's Name],</p>  

<p>I hope this message finds you well. My name is [Your Name], and I am a representative from BroomBot, where we specialize in providing innovative motorcycle services and products powered by advanced AI technology.</p>  

<p>We understand that choosing the right motorcycle service can be overwhelming, and I would like to offer our support. Whether you are interested in booking a motorcycle for your next adventure or exploring our range of motorcycles for sale, our team is here to assist you with any questions you may have.</p>  

<p>At BroomBot, we pride ourselves on delivering excellent service tailored to your needs. Our user-friendly platform makes booking seamless and efficient, ensuring you have the best experience possible.</p>  

<p>If you are open to discussing how we can assist you, please feel free to reply to this email or contact me directly at [Your Phone Number]. I would be delighted to help 

In [ ]:
customer_service_picker = Agent(
    name="customer_service_picker",
    instructions="You pick the best cold customer service email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [ ]:
message = "Write a cold customer service email"

with trace("Selection from customer service people"):
    results = await asyncio.gather(
        Runner.run(customer_service_agent1, message),
        Runner.run(customer_service_agent2, message),
        Runner.run(customer_service_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold customer service emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(customer_service_picker, emails)

    print(f"Best customer service email:\n{best.final_output}")


Best customer service email:
<!DOCTYPE html>  
<html lang='en'>  
<body>  
<p>Subject: Ready to Ride? Let BroomBot Help You Zoom!</p>  

<p>Hi [Recipient's Name],</p>  

<p>Are you tired of four wheels and feeling a bit wheelie bored? If you’re looking to kick the mundane to the curb and add some vroom to your zoom, then let me introduce you to BroomBot – your new best buddy on two wheels!</p>  

<p>With our AI-powered motorcycle booking service, you can ride into the sunset quicker than you can say “rev it up!” Whether you're planning a weekend getaway or just want to ditch the traffic jams, we’ve got your back with motorcycles that will make your heart race and your worries melt away.</p>  

<p>Need a ride? We offer easy booking with just a click, and our service is as smooth as a well-oiled machine. Want to own a slice of freedom? Check out our range of motorcycles that come with a warranty of fun!</p>  

<p>Ready to take the plunge? Just hit reply, and we’ll be your co-pilots in th

## Part 2: use of tools

Now we will add a tool to the mix.

Remember all that json boilerplate and the `handle_tool_calls()` function with the if logic..

In [ ]:
customer_service_agent1 = Agent(
        name="Professional Customer Service Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

customer_service_agent2 = Agent(
        name="Engaging Customer Service Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

customer_service_agent3 = Agent(
        name="Busy Customer Service Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
customer_service_agent1

Agent(name='Professional Customer Service Agent', instructions="You are a customer service agent working for BroomBot (Motorcycle Bot), a company that provides a service motorcycle, booking service, and selling the product of motorcycle, powered by AI. You write professional, serious cold emails. Return the email in HTML format, do not include ```html but with <!DOCTYPE html> and exclude the head and css, div, etc.For example, like this:  <!DOCTYPE html> <html lang='en'> <body> <p>...</p> </body> </html>", prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type

## Steps 2 and 3: Tools and Agent interactions

Remember all that boilerplate json?

Simply wrap your function with the decorator `@function_tool`

In [62]:
import AIRPALibrary
ailabs = AIRPALibrary.AIRPALibrary()

In [74]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all customer service prospects """
    
    email_to = 'randi.oktariarinanda@ai.astra.co.id'
    email_from = 'elvino.dwisaputra@ai.astra.co.id'
    subject = 'Test'
    ailabs.send_email(subject=subject, content=f"""{body}""", to=email_to, sender=email_from)
    return {"status": "success"}

### This has automatically been converted into a tool, with the boilerplate json created

In [75]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all customer service prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020785283EC0>, strict_json_schema=True, is_enabled=True)

### And you can also convert an Agent into a tool

In [ ]:
tool1 = customer_service_agent1.as_tool(tool_name="customer_service_agent1", tool_description="Write a cold customer service email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000207853E93A0>, strict_json_schema=True, is_enabled=True)

### So now we can gather all the tools together:

A tool for each of our 3 email-writing agents

And a tool for our function to send emails

In [ ]:
description = "Write a cold customer service email"

tool1 = customer_service_agent1.as_tool(tool_name="customer_service_agent1", tool_description=description)
tool2 = customer_service_agent2.as_tool(tool_name="customer_service_agent2", tool_description=description)
tool3 = customer_service_agent3.as_tool(tool_name="customer_service_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002078542F560>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002078542F2E0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'

## And now it's time for our Sales Manager - our planning agent

In [ ]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
- Send email body must be in HTML format, do not include ```html but with <!DOCTYPE html> and exclude the head and css, div, etc. But you can add like ul, li, etc based on the email content.
For example, like this:
<!DOCTYPE html> 
<html lang='en'> 
<body> 
<p>...</p> 
</body> 
</html>"
"""


customer_service_manager = Agent(name="Customer Service Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold customer service email addressed to 'Dear CEO'"

with trace("Customer Service manager"):
    result = await Runner.run(customer_service_manager, message)

{
  "message": "Email berhasil dikirim!",
  "success": true
}



In [80]:
result

RunResult(input="Send a cold sales email addressed to 'Dear CEO'", new_items=[ToolCallItem(agent=Agent(name='Customer Service Manager', instructions='\nYou are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold sales email using the sales_agent tools.\n\nFollow these steps carefully:\n1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.\n\n2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.\n\n3. Use the send_email tool to send the best email (and only the best email) to the user.\n\nCrucial Rules:\n- You must use the sales agent tools to generate the drafts — do not write them yourself.\n- You must send ONE email using the send_email tool — never more than one.\n- Send email body must be in HTML format, do not include ```html but with <!DOCTYPE html> and exclude the head and 

### Handoffs represent a way an agent can delegate to an agent, passing control to it

Handoffs and Agents-as-tools are similar:

In both cases, an Agent can collaborate with another Agent

With tools, control passes back

With handoffs, control passes across



In [81]:

subject_instructions = "You can write a subject for a cold customer service email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold customer service email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [82]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all customer service prospects """
    email_to = 'randi.oktariarinanda@ai.astra.co.id'
    email_from = 'elvino.dwisaputra@ai.astra.co.id'
    ailabs.send_email(subject=subject, content=f"""{html_body}""", to=email_to, sender=email_from)
    return {"status": "success"}

In [83]:
tools = [subject_tool, html_tool, send_html_email]

In [84]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000207854B1B20>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000207854B20C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HT

In [85]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [86]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002078542F560>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002078542F2E0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold customer service email', params_json_schema={'properties': {'input': {'ti

In [ ]:
# Improved instructions thanks to student Guillermo F.

customer_service_manager_instructions = """
You are a Customer Service Manager at BroomBot (Motorcycle Bot). Your goal is to find the single best cold customer service email using the customer_service_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three customer_service_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the customer service agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


customer_service_manager = Agent(
    name="Customer Service Manager",
    instructions=customer_service_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold customer service email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(customer_service_manager, message)